In [ ]:
Types of docs
* listings
    * different document types
* meetinsg
* reports

TODO during evaluation: representatieve documenten set

In [1]:
import numpy as np
import pandas as pd
import os
import xml.etree.ElementTree as ET
import pickle
import math
import requests
import shutil
from copy import deepcopy
from tqdm import tqdm

# Processing the meetings files

In [2]:
# setting the directory containing the meetings
meetings_dir = "../20210415/"

# setting the directory for where to save the document files
documents_dir = "../Documents/"

meetings = [s for s in os.listdir(meetings_dir) if "meetings_" in s]
listings = [s for s in os.listdir(meetings_dir) if "list_" in s]

print(listings)

['list_09864d46-d5ff-4d56-b30a-d28e4318d6b4.xml', 'list_1a44fa8b-87b1-4076-bcbc-2e4da459ad2b.xml', 'list_1dae7eda-d078-4d87-8f47-01d6ea8116b0.xml', 'list_1f88a645-6dde-4a79-b30a-0e96fde67465.xml', 'list_20283bf6-bf36-4408-a60f-bd63de8b6e63.xml', 'list_41b35454-6329-4f11-b803-d7e5a6141085.xml', 'list_50ba9ec6-f591-4070-8eea-4755d4189973.xml', 'list_77898070-498a-47ab-aca5-9ae61cdaf340.xml', 'list_8163a4ce-b771-44f8-a2e8-129bc5c628ce.xml', 'list_8274f276-27b9-4767-a053-e41df1f0d195.xml', 'list_a832e265-1b2f-4cac-b168-ab6915f4b09d.xml', 'list_ac711b72-54f7-449a-9a73-9693e6e06688.xml', 'list_b14b6247-aff1-4e57-97f1-9866cac20290.xml', 'list_b25f250f-d487-4ce5-93b3-efb84d24567a.xml', 'list_c2520e4e-0bbe-4e65-bda4-939f15b8d0b2.xml', 'list_df3db841-304a-4c26-9294-ff1bca4a694d.xml', 'list_fcf15b6c-0a82-4e7a-b405-f38656cf60f7.xml']


In [3]:
# PARSE MEETINGS

# list containing information about meetings
meetingsInformation = {}

# list containing all the (agenda) item information
itemsInformation = {}


# list containing all the document information
documentsInformation = {}

# list containing all the document information
meetdocumentsInformation = {}


found = 0
for file in meetings:
    tree = ET.parse(meetings_dir + file)
    root = tree.getroot()

    print('parsing ' + file)

    # loop through all meetings
    for meeting in root:

        # intermediary information for the meetings
        meetingInformation = {}

        meetingInformation["meeting id"] = meeting.attrib["id"]
        
        # list to store the (agenda) items in
        items = []
        docs = []
        
        # Go through all children of the meeting
        for child in meeting:
            if not child.tag == "item":
                meetingInformation[child.tag] = child.text

            # if child is an item go through all the children of the item and store their information
            if child.tag == "item":
                
                # append item to items
                items.append(child.attrib['id'])
                
                # intermediary information for the items
                itemInformation = {}
                itemInformation["item id"] = child.attrib["id"]
#                print(meeting)
#                itemInformation["Datum"] = meeting.attrib["Datum"]
                
                # list for saving all documents attached to this item
                documents = []
                
                # loop through all items
                for itemChild in child:
                    # now repeat once more to find the documents
                    if itemChild.tag == "document":
                        
                        # append this document to list of documents
                        documents.append(itemChild.attrib['id'])
                        
                        # intermediary information for the document
                        documentInformation = {}
                        documentInformation["document id"] = itemChild.attrib["id"]
                        documentInformation["origin id"] = child.attrib["id"]
                        documentInformation["origin type"] = "item"

                        for documentChild in itemChild:
                            #Why is this here? it never fires..
                            #is it because the author copied code from the parsing of agenda items?
                            if itemChild.text == "\n     ":
                                print('what is this?')
                                print(itemChild)
                                print()
                                documentInformation[documentChild.tag] = documentChild.attrib
                            else :
                                documentInformation[documentChild.tag] = documentChild.text


                        # store all information for this document
                        documentsInformation[itemChild.attrib["id"]] =  documentInformation
                        
                    #not a document? store attributes in agenda point
                    else:
                        itemInformation[itemChild.tag] = itemChild.text

                    

                # attach all documents belonging to this item
                itemInformation["Documents"] = documents
                itemInformation["origin id"] = meeting.attrib['id']
                itemInformation['parentid'] = ""
                
                
                
                #Check if this item has a parent (e.g. https://utrecht.bestuurlijkeinformatie.nl/Agenda/Index/6033aaab-fee5-416b-bf65-46b993cb6e40#86a46282-d1a9-4528-a6a9-9cd4c146cb8f)
                
                #first: what is an item without a features? is it the old data?
                #answer: it seems like raadsinformatiebijeenkomsten, besloten vergaderingen, ...
                #things we can safely ignore
                if 'features' in itemInformation:
                
                    #changed if we find the parent
                    itemInformation["parentid"] = ''
                    
                    #if it contains a . it is probably a child - make sure it doesnt end in a . e.g. 1.
                    #
                    if '.' in itemInformation['features'] and itemInformation['features'][-1] != '.':
                        #ignore meeting types 17 (mondelijke vragen), 100716987 (questions)
                        #TODO investigate if we should ignore voorjaarsnota's (and corresponding jaarstukken)
                        if str(meetingInformation['type']) != '17' and str(meetingInformation['type']) != '100716987':
                            #print('type ' + meetingInformation['type'] + ' itemid ' + str(itemInformation['features']) + ' ' + str(meeting.attrib['id']))
                            #print('    ' + itemInformation['title'])
                            
                            #find parent id
                            # if there is a parent, find parent + .
                            # if we group on the first child, check for (6.)4 by splitting on .
                            if (parentFeature.split('.')[0]) + '.' in itemInformation['features']:
                                itemInformation["parentid"] = parentId
                            
                                #we found a parent! let's merge this item with the parent
                                #TODO change I guess.. now we lose entryIds, which we use to .. 
                                for d in itemInformation["Documents"]:
                                    itemsInformation[parentId]["Documents"].append(d)
                                    #print(d)
                                    #print(itemsInformation[parentId]["Documents"])
                            
 #                           elif (parentFeature.split('.')[0]) + '.' in itemInformation['features']:
                                
                            else:
                  #              print(parentFeature)
                  #              print(itemInformation['features'])
                  #              print()
                                #we have no parent, but we'll be parents to the next
                                parentId = itemInformation['item id']
                                parentFeature = itemInformation['features']
                                
                                #case: no (known) parent. how should we group them?
                            
                            #if it is a child, dont update the parent id: the previous parent might have multiple children
                            found += 1

                    else:
                        #readability
                        if found > 0:
                            #print()
                            found = 0
                        #store for the next item we parse 
                        parentId = itemInformation["item id"]
                        parentFeature = itemInformation["features"]
#                else:
#                    print(itemInformation)
                                     
                # store all information for this item 
                itemsInformation[child.attrib["id"]] = deepcopy(itemInformation)
                
                
                
            elif child.tag == 'document':
                # append this document to list of documents
                docs.append(child.attrib['id'])
                        
                # intermediary information for the document
                docInformation = {}
                docInformation["document id"] = child.attrib["id"]
                docInformation["origin id"] = meeting.attrib["id"]
                docInformation["origin type"] = "meeting"

                for docChild in child:
                    #Why is this here? it never fires..
                    #is it because the author copied code from the parsing of agenda items?
                    if docChild.text == "\n     ":
                            print('what is this?')
                            print(docChild)
                            print()
                            docInformation[docChild.tag] = docChild.attrib
                    
                    else :
                        docInformation[docChild.tag] = docChild.text
                        # store all information for this document
                        docInformation[child.attrib["id"]] = docInformation

                    # store all information for this document
                    meetdocumentsInformation[child.attrib["id"]] = deepcopy(docInformation)
                    
                        
            
            # save the items attached to this meeting
            meetingInformation["items"] = items
            meetingInformation["documents"] = docs
         # store all information for this meeting
        meetingsInformation[meeting.attrib["id"]] = meetingInformation
        
# Sanity check
print()
print(str(len(meetingsInformation)) + " meetings parsed")
print(str(len(itemsInformation)) + " items parsed")

print()
print(str(len(meetdocumentsInformation)) + " meeting documents parsed")
print(str(len(documentsInformation)) + " agenda item documents parsed")

# meeting type?

#H1: if 6.1 6.2 etc... AND 6  exists, then group
    #no sometimes we still want to group I guess...

#TODO search for "parentname."  in childname


#TODO test MV.2 MV3 etc
#TODO f3d08622-7a45-475b-afc9-68e96ab49b22   - lijst documenten (raadsbrieven)
#TODO 6b248a0f-6f33-4663-a99f-570975bd80f2   - lijst documenten (schriftelijke vragen)



#LATEN WE VOORBEELDEN VERZAMELEN DIE WE MOETEN FILTEREN
# f3d08622-7a45-475b-afc9-68e96ab49b22     - voorjaarsnota:  jaarstukken en relevante raadsvoorstellen?     meeting type 2
    #JUIST WEL OF JUIST NIET GROEPEREN? Ik kan me voorstellen 
    #WEL als je voorstel jaarstukken bekijkt
    #NIET als je naar zo'n raadsstuk kijkt
    #-> Het is dan niet een transitieve eigenschap: deelpunten alleen groeperen als je naar context hoofdpunt kijkt?
    # edgecase: ook voorbeelden waar geen hoofdpunt is maar de deelpunten wel gegroepeerd meoten worden...
    #  -> geldt deze regel alleen voor jaarstukken?
    
# 61c08ca4-2f57-474b-8a7a-0b398e091ccb  - verzameling schriftelijk vragne antwoorden    type 0     geen hoofdpunt   !
# fd864cf8-6ecc-4c73-825d-fcc0a848a27f  - idem      !
# 94e21e8f-6977-4883-9aca-403ba7f13f10  idem


# d2eff14d-7735-4e88-83a6-ce11b1b8d76f   jaarstukken   type 2  !
# 94e21e8f-6977-4883-9aca-403ba7f13f10     idem twijfelgeval
# 20ecac3a-b8f5-4817-b7c9-d55904e7a5d1
# a1b22807-377a-46b8-bd23-9afd5ba3540b

# 45aac33c-d537-47e4-af63-1f1c1edfd359  voorjaarsnota !

# 429140f0-24d2-4ca5-a83b-f3577baa29cc  vragenuur type 17    !
# c2b2c62c-e387-4723-a19f-3e4a1cfbad85  vragenuur type 17


#b8bb8994-3f4a-4f70-b8f4-8d4901a7ea01   twijfelgeval. verzamelt armoede aanpak dingen. handig als je 

#ea94b4ad-ee17-447d-aa25-81fd1aea8bdf
# 17977cce-f828-400f-84cc-958dd5ba896e   twijfel

#9fa8e966-afc5-4d64-8f4f-e7fe9301ffba   stemming 100716987 
#3e0ed66f-462c-46f0-98d5-dbb4948e57dc    type 100716987   stemagenda raad 
#f8b3c018-31ae-47ec-b7f3-2f2f7def73f0  100716987 
#itemid 6.1.1 55473b4b-31fb-45ab-a677-1b7124d52060 100716987
#a8e676cf-e88c-4999-854b-3ebd77f185b6  100716987
#0a997446-f6ab-4d59-8695-ac634dae0284 100716987



# twijfel 2a00092d-37bc-487b-9ffc-54b843bf1312

#test GROEPEERT  deze logisch  e7496e5e-5dbe-4327-8e98-1443a1ac903a

#TEST groeperen we deze goed?
# 61c08ca4-2f57-474b-8a7a-0b398e091ccb   6b248a0f-6f33-4663-a99f-570975bd80f2

#TEST  groepeert deze dingen die er niet bijhoren? 738a36af-5631-4661-80ce-b9a84fef9c36

# TESt gaat deze over rattenoverlast nu op onhanidge manier alles over afval in deze wijk combineren ? e6f6c560-3fe2-4231-bd50-4712a368c295

#0cbe97fe-129b-4b70-b914-9b29dc9e4ad6  goed voor testen - groepeert SVs moties enzo


#H2   je kan ook kijken hoeveel tijdlijnen een agendapunt zou mergen , en het niet mergen als het teveel zou doen
#H3   je wil alleen kijken naar waar een agendapunt eerder is gebruikt  - je wil de toekomst niet in kijken om ze te mergen

parsing meetings_2000_1.xml
parsing meetings_2000_10.xml
parsing meetings_2000_11.xml
parsing meetings_2000_2.xml
parsing meetings_2000_3.xml
parsing meetings_2000_4.xml
parsing meetings_2000_5.xml
parsing meetings_2000_6.xml
parsing meetings_2000_8.xml
parsing meetings_2000_9.xml
parsing meetings_2001_1.xml
parsing meetings_2001_10.xml
parsing meetings_2001_11.xml
parsing meetings_2001_12.xml
parsing meetings_2001_2.xml
parsing meetings_2001_3.xml
parsing meetings_2001_4.xml
parsing meetings_2001_5.xml
parsing meetings_2001_6.xml
parsing meetings_2001_8.xml
parsing meetings_2001_9.xml
parsing meetings_2002_1.xml
parsing meetings_2002_10.xml
parsing meetings_2002_11.xml
parsing meetings_2002_3.xml
parsing meetings_2002_4.xml
parsing meetings_2002_5.xml
parsing meetings_2002_6.xml
parsing meetings_2002_9.xml
parsing meetings_2003_1.xml
parsing meetings_2003_10.xml
parsing meetings_2003_12.xml
parsing meetings_2003_2.xml
parsing meetings_2003_3.xml
parsing meetings_2003_4.xml
parsing mee

In [4]:
#PARSE LISTINGS

# Each list is a list of Entries. An entry may contain multiple documents
# e.g. a council letter with multiple attachments

listings = [s for s in os.listdir(meetings_dir) if "list_" in s]


# list containing information about entries
entriesInformation = {}

# list containing all the listdocument information
listdocumentsInformation = {}

for file in listings:
    
    tree = ET.parse(meetings_dir + file)
    root = tree.getroot()

    #this list.xml contains the following document type
    doctype = root.attrib['name']
    print('parsing ' + doctype)
    
    # Go through all documents in the list
    for entry in root:
        
        # information about the entry
        entryInformation = {
                "entry id": entry.attrib["id"],
                "doctype": doctype
        }
        
        # map entry id to doc ids
        entryDocs = []
        
        # Go through all children of the meeting
        for child in entry:
            
            
            # Each child is metadata or a document
            if child.tag == "document":
                entryDocs.append(child.attrib['id'])
                
                # intermediary information for the document
                documentInformation = {}
                documentInformation["document id"] = child.attrib["id"]
                documentInformation["origin id"] = entry.attrib["id"]
                documentInformation["origin type"] = "entry"

                #copy all document attributes. if empty tag, put attribute name
                for docAttrib in child:

                    #legacy I guess? not sure why this is here - it never fires the If condition
                    #from parsing meetings?
                    if child.text == "\n     ":
                        print('manual check - is this just an empty tag?')
                        print(child)
                        documentInformation[docAttrib.tag] = docAttrib.attrib
                    else :
                        documentInformation[docAttrib.tag] = docAttrib.text
                
                # store this document in full list of documents
                # deepcopy just to be sure we don't change the same documentInformation later
                listdocumentsInformation[child.attrib["id"]] = deepcopy(documentInformation)
                
                
            #copy entry metadata
            elif child.tag == "property":
                entryInformation[child.attrib['key']] = child.text
                
            # debug: if it's not a document or property, what is it? 
            else:
                print("What is this? " + str(child.tag))
                #if itemChild.text == "\n        " and not child.tag == "document":

                
        # After processing an entry's children
            
        # Update related doc ids for this entry
        entryInformation["Documents"] = entryDocs
            
        # store this entry to list of entries
        entriesInformation[entry.attrib["id"]] = deepcopy(entryInformation)
            
            
# Sanity check
print(str(len(entriesInformation)) + " entries parsed")
print(str(len(listdocumentsInformation)) + " documents parsed")

parsing Verslagen M&S
parsing Schriftelijke vragen
parsing Verslagen db Presidium
parsing Verslagen Vragenuur
parsing Verslagen S&R
parsing Brieven 
parsing Amendementen
parsing Archief
parsing Geheimhoudingsregister
parsing Intentiedocumenten
parsing Vragen Coronavirus
parsing Memo's
parsing Verslagen Raad
parsing Moties
parsing Stemming
parsing Verslagen Presidium
parsing Besluitenlijst B&W
19156 entries parsed
22908 documents parsed


In [5]:
with open("meetingsInformation.txt", "wb") as fp:
    pickle.dump(meetingsInformation, fp)
with open("itemsInformation.txt", "wb") as fp:
    pickle.dump(itemsInformation, fp)
with open("entriesInformation.txt", "wb") as fp:
    pickle.dump(entriesInformation, fp)    
    
with open("listdocumentsInformation.txt", "wb") as fp:
    pickle.dump(listdocumentsInformation, fp)       
with open("meetdocumentsInformation.txt", "wb") as fp:
    pickle.dump(meetdocumentsInformation, fp)
with open("documentsInformation.txt", "wb") as fp:
    pickle.dump(documentsInformation, fp)    


In [6]:
#Wait - do we even need this? Download the dataset


# setting the main file path to where all files are
os.chdir("C:\\Users\\tmsch\\Desktop\\ADS_Thesis\\PDFs")

# setting the directory for where to save the document files
documents_dir = "PDFs/Documents/"

for doc in documentsInformation:
    r = requests.get(doc['publicdownloadurl'])


#listdocumentsInformation and meetdocumentsinformation

TypeError: string indices must be integers

# (Near) duplicate detection

In [7]:
# First we construct a testset based on
#  all sets of documents with the same filename (of at least size 10)
#  all sets of documents with the same displayname (of at least size 10)
#  all sets of documents with the same filesize (of at least size 10)


#blacklist = ['Lijst van ingekomen stukken.pdf',
#            'Agendabundel.pdf',
#             
#]



def addDocuments(docs, uniques, key):
    for doc in docs:
        newdoc = {
            'id': doc,
            'filename': docs[doc]['filename'],
            'displayname': docs[doc]['displayname'],
            'filesize': docs[doc]['filesize'],
            'publicdownloadurl': docs[doc]['publicdownloadurl'],
            'origin id': docs[doc]['origin id'],
            'origin type': docs[doc]['origin type']
        }
                
        if newdoc[key] in uniques:
            uniques[newdoc[key]].append(deepcopy(newdoc))
        else:
            uniques[newdoc[key]] = [deepcopy(newdoc)]
            
        

def printDupes(unique, length, v):
    count = 0
    for doc in unique:
        l = len(unique[doc])
        if l > length:
            count += 1
            print(doc)
            if v == 'verbose':
                print()

                for dname in unique[doc]:
                    print(dname['filename'] + '    ' + dname['displayname'])
                print()
                print()
                print()
    print(str(count) + ' sets with 10+')

def writeTestset(unique):
    count = 0
    for doc in unique:
        l = len(unique[doc])
        if l > 10:
            with open('C:\\Users\\tmsch\\Desktop\\ADS_Thesis\\Thomas\\testset2.csv','a') as file:
                for dname in unique[doc]:
                    file.write(dname['filename'] + ',' + dname['displayname'] + ',' + dname['filesize'] + ',' + dname['publicdownloadurl'] + '\n')
            count += 1
#            print(doc)

    
    
uniquedocs = {}
uniquedisp = {}
addDocuments(documentsInformation, uniquedocs, 'filename')
addDocuments(listdocumentsInformation, uniquedocs, 'filename')
#addDocuments(meetdocumentsInformation, uniquedocs, 'filename')

#COMPLETENESS METRIC
#How many filenames were descriptive and how many were not

addDocuments(documentsInformation, uniquedisp, 'displayname')
addDocuments(listdocumentsInformation, uniquedisp, 'displayname')
#printDupes(uniquedisp)
#addDocuments(meetdocumentsInformation, uniquedisp, 'displayname')

uniquesize = {}
addDocuments(documentsInformation, uniquesize, 'filesize')
addDocuments(listdocumentsInformation, uniquesize, 'filesize')
#printDupes(uniquesize)


#print(uniquedisp[list(uniquedisp)[0]])
#printDupes(uniquedocs)
#writeTestset(uniquedocs)
#writeTestset(uniquedisp)

#uniquedisp = {}
#addDocuments(documentsInformation, uniquedisp, 'displayname')
#printDupes(uniquedisp)

#uniquesize = {}
#addDocuments(documentsInformation, uniquesize, 'filesize')
#printDupes(uniquesize)
            

#What if we group on filesize instead? of displayname?


#addDisplay(listdocumentsInformation)
#addDisplay(meetdocumentsInformation)
#print(len(unique_display))
#printDupes(unique_display)

#Seems like we can just combine them - the only overlapping filename is 1.pdf
#addDocuments(documentsInformation)
#addDocuments(listdocumentsInformation)
#addDocuments(meetdocumentsInformation)
#printDupes()



#Make testset with all items with 10+ dupes



In [8]:
#Let's try to find dupes!
keys1 = []
keys2 = []
keys2fnames = []

uniqueses = {}


keys3 = []
from time import sleep

def count_uniques(lst):
    full = []
    
    #print(lst)
    for l in lst:
        if l['id'] not in full:
            full.append(l['id'])
            
    return len(full)

#deprecated
def filter_nondupes(u):
    u2 = {}
    
    for unique in u:
        if len((u[unique])) > 1:
            u2[unique] = deepcopy(u[unique])

from time import sleep 
def filter_nondupes_u(u):
    print('BEFORE ')
    print(len(set(u)))
    u2 = {}
    for unique in u:
        if count_uniques(u[unique]) > 1:
            #print(u[unique])
            #sleep(1)
            u2[unique] = deepcopy(u[unique])
    u = deepcopy(u2)
    print('AFTER ')
    print(len(set(u)))
    return u

def uniques_name(docs, u):
    u2 = deepcopy(u)
    
    for doc in docs:
        newdoc = {
            'id': doc,
            'filename': docs[doc]['filename'],
            'displayname': docs[doc]['displayname'],
            'filesize': docs[doc]['filesize'],
            'publicdownloadurl': docs[doc]['publicdownloadurl'],
            'origin type': docs[doc]['origin type'],
            'origin id': docs[doc]['origin id']
        }

        key = (newdoc['filename'].strip() + ' ' + newdoc['filesize'].strip()).replace('.pdf', '').replace('.docx','').replace('.doc','').replace('.docx','').replace('.doc','')
        
        if key in u2:
            u2[key].append(deepcopy(newdoc))
        else:
            u2[key] = [deepcopy(newdoc)]

    for unique in u2:
        #if len(u2[unique]) > 1:
            global uniqueses
            u[unique] = deepcopy(u2[unique])
            uniqueses[unique] = deepcopy(u2[unique])
            keys1.append( ( u[unique][0]['filename'].strip() + ' ' + u[unique][0]['filesize'].strip()).replace('.pdf','').replace('.docx','').replace('.doc',''))
            


def uniques_disp(docs, u):
    u2 = deepcopy(u)
    
    for doc in docs:
        newdoc = {
            'id': doc,
            'filename': docs[doc]['filename'],
            'displayname': docs[doc]['displayname'],
            'filesize': docs[doc]['filesize'],
            'publicdownloadurl': docs[doc]['publicdownloadurl'],
            'origin type': docs[doc]['origin type'],
            'origin id': docs[doc]['origin id']
        }
                
        key = (newdoc['displayname'].strip() + ' ' + newdoc['filesize'].strip()).replace('.pdf', '').replace('.docx','').replace('.doc','')
        

        if key in u2:
            u2[key].append(deepcopy(newdoc))
        else:
            u2[key] = [deepcopy(newdoc)]
            
    for unique in u2:
        #if len(u2[unique]) > 1:
            u[unique] = deepcopy(u2[unique])

            key2 = (u[unique][0]['displayname'].strip() + ' ' + u[unique][0]['filesize'].strip()).replace('.pdf','').replace('.docx','').replace('.doc','')
            keys2.append(key2)
            
            global uniqueses
            
            #add this to the complete array of uniques - I guess we'll group it on the filename of the first document
            #If one of them shares a filename, they would be grouped anyway. Otherwise, it is a unique id
            if key2 in uniqueses:                
                uniqueses[key2].extend(deepcopy(u2[unique]))
            else:
                uniqueses[key2] = deepcopy(u2[unique])
            
            fnames = []
            for d in u[unique]:
                fnames.append( ( d['filename'].strip() + ' ' + d['filesize'].strip()).replace('.pdf','').replace('.docx','').replace('.doc',''))

            #uniqueses[key2]
            keys2fnames.append(deepcopy(fnames))
            
            
def uniques_name_disp(docs, u):
    u2 = deepcopy(u)
    
    for doc in docs:
        newdoc = {
            'id': doc,
            'filename': docs[doc]['filename'],
            'displayname': docs[doc]['displayname'],
            'filesize': docs[doc]['filesize'],
            'publicdownloadurl': docs[doc]['publicdownloadurl'],
            'origin type': docs[doc]['origin type'],
            'origin id': docs[doc]['origin id']
        }
                
        key = (newdoc['filename'].strip() + ' ' + newdoc['displayname'].strip()).replace('.pdf', '').replace('.docx','').replace('.doc','')
        
        if key in u2:
            u2[key].append(deepcopy(newdoc))
        else:
            u2[key] = [deepcopy(newdoc)]

    for unique in u2:
        #if len(u2[unique]) > 1:
            filesizes = []
            for d in u2[unique]:
                filesizes.append(int(d['filesize']))
            avgsize = sum(filesizes) / len(filesizes)
            
            trust = 1
            for d in u2[unique]:
                if not (int(d['filesize']) > (avgsize - 0.5 * avgsize) and int(d['filesize']) < (avgsize + 0.5 * avgsize)):
                    trust = 0
            
            
            if trust:         
                global uniqueses
                u[unique] = deepcopy(u2[unique])
                key3 = ( u[unique][0]['filename'].strip() + ' ' + u[unique][0]['filesize'].strip()).replace('.pdf','').replace('.docx','').replace('.doc','')
                
                if key3 in uniqueses:
                    uniqueses[key3].extend(deepcopy(u2[unique]))
                else:
                    uniqueses[key3] = deepcopy(u2[unique])
                    
                keys3.append( key3 )
            

            
uniques1 = {}
uniques_name(documentsInformation, uniques1)
uniques_name(listdocumentsInformation, uniques1)
filter_nondupes(uniques1)
#printDupes(uniques, 5, 'non-verbose')
print()

uniques2 = {}
uniques_disp(documentsInformation, uniques2)
uniques_disp(listdocumentsInformation, uniques2)
filter_nondupes(uniques2)
#printDupes(uniques, 5, 'non-verbose')
print()

uniques3 = {}
uniques_name_disp(documentsInformation, uniques3)
uniques_name_disp(listdocumentsInformation, uniques3)
filter_nondupes(uniques3)
#printDupes(uniques, 5, 'non-verbose')


uniqueses = filter_nondupes_u(uniqueses)
print(len(set(uniqueses)))

uniques4 = {}
#try on filesize?


#print(uniques.keys())
print('Basic statistics on how many duplicates we find')
print()
print('uniqueses')
print(len(set(uniqueses)))
print('How many unique filenames in sets 1 and 2?')
print(len(set(keys1)))
print(len(uniques1))
print()
print(len(set(keys2)))
print(len(uniques2))

print('How many of those overlap?')
print(len(set(keys1) & set(keys2)))

count = 0
for fnames in keys2fnames:
    present = 0
    for fname in fnames:
        if fname.strip().replace('.pdf','').replace('.docx','').replace('.doc','') in keys1:
            present += 1
    if present > 0:
        count += 1

print(count)

#print(len(set(keys1) & set(keys2filenames)))


print()
print('How many with same filename/displayname?')
print('unique filenames')
print(len(set(keys3)))
print('filenames')
print(len(keys3))
print('unique filename/displayname combos')
print(len(uniques3))

#print()
#print()
#printDupes(uniques3, 10, 'nonverbose')


print()
print()
print('How much overlap with set1?')
count = 0
for key in uniques3:
    present = 0
    for d in uniques3[key]:
#        print (d['filename'].strip().replace('.pdf',''))
        if (d['filename'].strip() + ' ' + d['filesize'].strip()).replace('.pdf','').replace('.docx','').replace('.doc','') in keys1:
            present += 1
    if present > 0:
        count += 1
print(count)
    
print('So we got lots of new items! Lets check if the precision is high/results are reliable')
for key in uniques3:
    if len(uniques3[key]) > 2:
        print(key)
        print()
        for d in uniques3[key]:
            print(d['filename'] + ' ' + d['filesize'])
        print()
        print()



BEFORE 
54543
AFTER 
6320
6320
Basic statistics on how many duplicates we find

uniqueses
6320
How many unique filenames in sets 1 and 2?
49144
49144

49254
49254
How many of those overlap?
43855
76171

How many with same filename/displayname?
unique filenames
43404
filenames
67248
unique filename/displayname combos
43507


How much overlap with set1?
43507
So we got lots of new items! Lets check if the precision is high/results are reliable
Bijlage 1 Aanvullend advies van der Poel Bijlage 1 Aanvullend advies van der Poel

Bijlage 1 Aanvullend advies van der Poel.pdf 55959
Bijlage 1 Aanvullend advies van der Poel.pdf 55956
Bijlage 1 Aanvullend advies van der Poel.pdf 55959
Bijlage 1 Aanvullend advies van der Poel.pdf 55959
Bijlage 1 Aanvullend advies van der Poel.pdf 55959
Bijlage 1 Aanvullend advies van der Poel.pdf 55959
Bijlage 1 Aanvullend advies van der Poel.pdf 55959
Bijlage 1 Aanvullend advies van der Poel.pdf 55959
Bijlage 1 Aanvullend advies van der Poel.pdf 55959
Bijlage 1 

Dossier 734 voorblad.pdf 18800
Dossier 734 voorblad.pdf 18848


Raadsbrief Voorstel nieuwe doelstellingen en indicatoren programma Cultuur Raadsbrief Voorstel nieuwe doelstellingen en indicatoren programma Cultuur

Raadsbrief Voorstel nieuwe doelstellingen en indicatoren programma Cultuur.pdf 32070
Raadsbrief Voorstel nieuwe doelstellingen en indicatoren programma Cultuur.pdf 34430
Raadsbrief Voorstel nieuwe doelstellingen en indicatoren programma Cultuur.pdf 29972


Vernieuwde doelstellingen en indicatoren voor het programma Cultuur Vernieuwde doelstellingen en indicatoren voor het programma Cultuur

Vernieuwde doelstellingen en indicatoren voor het programma Cultuur.pdf 74381
Vernieuwde doelstellingen en indicatoren voor het programma Cultuur.pdf 76762
Vernieuwde doelstellingen en indicatoren voor het programma Cultuur.pdf 72354
Vernieuwde doelstellingen en indicatoren voor het programma Cultuur.pdf 112554


Dossier 727 voorblad Dossier 727 voorblad

Dossier 727 voorblad.pdf 19152
Do

In [9]:
"""#Test if we look for overlaps entries/items 

this blcok affects uniqueses...
t1 = {}
t2 = {}
uniques_name(documentsInformation, t1)
uniques_name(listdocumentsInformation, t2)

t3 = {}
uniques_name(documentsInformation, t3)
uniques_name(listdocumentsInformation, t3)
filter_nondupes(t3)

print(len(uniqueses))



#print(len(uniques1))
#print(len(t1))
#print(len(t2))
#print(len(t3))
#print(len(listdocumentsInformation))
#print(len(entriesInformation))

count = 0
for i in t2:
    #doesnt fire
    if i in t1:
        print(i)
        count += 1
        
print(count)
"""
        
# test: neem voorbeeld uti hierboven en test of uniqueses 1 reference to entry en 1 reference to item?
#Raadsbrief Ter inzage legging stukken Uithoflijn 273482
#Bijlage bij raadsbrief - Statenbrief over Uithoflijn (2 maart 2018) 377406
#Auditrapport Uithoflijn 2017 (geanonimiseerd) 1515811
#Bijlage Hoofdlijnen Verbeterplan Uithoflijn Horvat 177514
#Raadsbrief Kwartaalrapportage Werkend Tramvervoersysteem Uithoflijn 2019 2e kwartaal 51589
#Kwartaalrapportage 2e kwartaal Werkend Tramvervoersysteem Uithoflijn 475188

'#Test if we look for overlaps entries/items \n\nthis blcok affects uniqueses...\nt1 = {}\nt2 = {}\nuniques_name(documentsInformation, t1)\nuniques_name(listdocumentsInformation, t2)\n\nt3 = {}\nuniques_name(documentsInformation, t3)\nuniques_name(listdocumentsInformation, t3)\nfilter_nondupes(t3)\n\nprint(len(uniqueses))\n\n\n\n#print(len(uniques1))\n#print(len(t1))\n#print(len(t2))\n#print(len(t3))\n#print(len(listdocumentsInformation))\n#print(len(entriesInformation))\n\ncount = 0\nfor i in t2:\n    #doesnt fire\n    if i in t1:\n        print(i)\n        count += 1\n        \nprint(count)\n'

In [10]:
#Test if meeting items have obvious duplicates
#IF SO there is a todo here - check out if we should merge them
    #result seems to be: not really.. so we do nothing

uniques = {}
for entry in itemsInformation:
    t = itemsInformation[entry]['title']
    if t in uniques:
        uniques[t].append(deepcopy(itemsInformation[entry]))
    else:
        uniques[t] = [deepcopy(itemsInformation[entry])]
    #print(t['description'])
print(len(uniques))

count = 0
for u in uniques:
    if len(uniques[u]) > 5:
        count += 1
        print(u)
        
print(count)
#        print(str(u) + ' ' + str(len(uniques[u])))
#        print()
#        for d in uniques[u]:
#            print(d)

8993
Lijst van mededelingen
Ingekomen stukken
Vragenuurtje
Agenda vaststellen en inventarisatie
Agenda vaststellen en inventariseren
Vaststellen agenda 
Werkafspraken tussen commissie en griffie
Opening en mededelingen 
Ingekomen stukken 
Toezeggingenlijst wethouder Janssen 
Terugkoppeling uit de gemeenschappelijke regelingen/verbonden partijen
Terugkoppeling uit de gemeenschappelijke regelingen/verbonden partijen 
Opening en mededelingen
Beantwoording rondvraagpunten wethouder Janssen
Beantwoording rondvraagpunten wethouder De Weger
Beantwoording rondvraagpunten wethouder mw. Spit
Beantwoording rondvraagpunten wethouder mw. Den Besten
Werkafspraken tussen commissie en griffie 
Vaststellen agenda
Toezeggingenlijst wethouder Janssen
Toezeggingenlijst wethouder De Weger
Mogelijkheid tot het stellen van rondvraagpunten aan wethouder Bosch
Beantwoording rondvraagpunten wethouder Bosch
Toezeggingenlijst wethouder Bosch
Vaststellen agenda en inventarisatie te bespreken agendapunten 
Beantwoo

In [11]:
#print(entryInformation)
#for item in itemsinformation:
#    print(meetingsInformation[list(itemsInformation)[0]['origin id']])
print(len(uniqueses))

6320


In [12]:
#Now let's find all agenda points that belong to dupes!



#TODO add date of meeting ot itemInofmration
dossiers = []
for u in uniqueses:
    

#    exists = False
#    for d in dossiers:
#        #TODO make sure u is the unique name
#        if u in d['filenames']:
#            add
#            exists = True
#    if not exists:
            #create new dossier
            dossier = {
                'filenames': [u],
                'doc_ids': [],
                'item_ids': [],
                'entry_ids': [],
                'origin_ids':[],
                'origin_types':[],
                'timeline': {}
            }
            
            for x in uniqueses[u]:
                #print(xu)
                #for x in xu:
                    dossier['doc_ids'].append(x['id'])
                    dossier['origin_ids'].append(x['origin id'])
                    dossier['origin_types'].append(x['origin type'])

                    #print(len(dossier['doc_ids']))
                    #print(len(dossier['origin_ids']))
                    #print()
                    
                    if x['origin type'] == 'item':
                        dossier['item_ids'].append(x['origin id'])

                        item = itemsInformation[x['origin id']]
                        #print(meetingsInformation[item['origin id']]['meetingdate'])

                        #NOTE meetingdate and publishdate are different
                        #publishdate seems more accurate with test data (origin publish instead of re upload date)

                        if meetingsInformation[item['origin id']]['meetingdate'] in dossier['timeline']:
                            dossier['timeline'][meetingsInformation[item['origin id']]['meetingdate']].append([x['origin id'], 'item'])
                        else:
                            dossier['timeline'][meetingsInformation[item['origin id']]['meetingdate']] = [[x['origin id'], 'item']]
                        #for doc in item['Documents']:
                        #    dossier['doc_ids'].append(doc)

                    elif x['origin type'] == 'entry':
                        dossier['entry_ids'].append(x['origin id'])

                        entry = entriesInformation[x['origin id']]

                        #print(entry)
                        key = ''
                        if 'Datum' in entry:
                            key = 'Datum'
                        elif 'Datum invoer' in entry:
                            key = 'Datum invoer'
                        else: 
                            key = 'Ontvangstdatum'

                        if entry[key] in dossier['timeline']:
                            dossier['timeline'][entry[key]].append([x['origin id'], 'entry'])
                        else:
                            dossier['timeline'][entry[key]] = [[x['origin id'], 'entry']]
                        #dossier['doc_ids'].append(doc)

                    else:
                        print('Origin type')
                        print(x)
            
            dossiers.append(deepcopy(dossier))

#print(documentsInformation[list(documentsInformation)[0]])
print(len(dossiers))
print(len(uniqueses))
#for d in dossiers:
#    print(len(d['doc_ids']))
#    print(len(d['origin_ids']))
#    print()

6320
6320


In [13]:
#Now go a step further: combine the different agenda points
udos = []
count = 0


#def addItem(item, uds):
    
#def addEntry(entry, uds):
    
#def addDocs(item, uds):

#def addDocuments(d):
    #First check all the documents attached to entries 
#    for entry in d['entry_ids']:
#        for doc in entriesInformation[entry]['Documents']:
#            if doc not in d['doc_ids']:
#                d['doc_ids'].append(doc)
#                d['filenames'].append((documentsInformation[doc]['filename'].strip() + ' ' + documentsInformation[doc]['filesize'].strip())
                #check if the new documents have duplicates
                

                
    #Now check all the documents attached to items
    
    
    
def mergeDossiers(d, ud):
    ud['filenames'] = set(list(ud['filenames']) + list(d['filenames']))
    ud['doc_ids'] = set(list(ud['doc_ids']) + list(d['doc_ids']))
    ud['item_ids'] = set(list(ud['item_ids']) + list(d['item_ids']))
    ud['entry_ids'] = set(list(ud['entry_ids']) + list(d['entry_ids']))
    
    ud['origin_ids'] = ud['origin_ids'] + d['origin_ids']
    ud['origin_types'] = ud['origin_types'] + d['origin_types']
    
    for time in d['timeline']:
        if time in ud['timeline']:   
            for t in d['timeline'][time]:
                ud['timeline'][time].append(t)
        else: 
            ud['timeline'][time] = deepcopy(d['timeline'][time])
            
    return ud
    
for d in tqdm(dossiers):    
    #check if this dossier should be merged
    found = False
    for i, ud in enumerate(udos):
        for docid in d['doc_ids']:
            if docid in ud['doc_ids']:
                found = True
                count += 1
                
                udos[i] = mergeDossiers(d, ud)
                #addDocuments(ud)
    if not found:
        udos.append(deepcopy(d))  #addDocuments
        
            
    #expand the current docs with other docs in the entry/docid files 
    #set a limit?
    # we do this afterwards: can we ever find a document that would make us merge again?
    # I don't think so - it should be atransitive /bidirecteional aspect
print(len(udos))
print(count)

100%|█████████████████████████████████████████████████████████████████████████████| 6320/6320 [00:25<00:00, 244.75it/s]

6035
1560


In [14]:
# lets test if mergedossiers is working out for us 
test = dossiers[0]
test2 = dossiers[1]
print(test)
print()
print(test2)
print()
test2 = mergeDossiers(test, test2)
print(test2)
print()

{'filenames': ['Bijlage 1 Programmabegroting 2007 5099105'], 'doc_ids': ['916ea22e-4016-45a2-ac43-2e0099706d72', '916ea22e-4016-45a2-ac43-2e0099706d72', '916ea22e-4016-45a2-ac43-2e0099706d72', '916ea22e-4016-45a2-ac43-2e0099706d72', '6c3b5566-8ecc-40ff-8ec3-71d0ff26470c', '916ea22e-4016-45a2-ac43-2e0099706d72', '6c3b5566-8ecc-40ff-8ec3-71d0ff26470c'], 'item_ids': ['be31f268-86db-42f8-985e-45d4a1147887', 'be31f268-86db-42f8-985e-45d4a1147887', 'be31f268-86db-42f8-985e-45d4a1147887', 'be31f268-86db-42f8-985e-45d4a1147887', '9b3af5fd-e90f-48a7-b90a-a5e5399ac70c', 'be31f268-86db-42f8-985e-45d4a1147887', '9b3af5fd-e90f-48a7-b90a-a5e5399ac70c'], 'entry_ids': [], 'origin_ids': ['be31f268-86db-42f8-985e-45d4a1147887', 'be31f268-86db-42f8-985e-45d4a1147887', 'be31f268-86db-42f8-985e-45d4a1147887', 'be31f268-86db-42f8-985e-45d4a1147887', '9b3af5fd-e90f-48a7-b90a-a5e5399ac70c', 'be31f268-86db-42f8-985e-45d4a1147887', '9b3af5fd-e90f-48a7-b90a-a5e5399ac70c'], 'origin_types': ['item', 'item', 'item'

In [15]:
print()

#Let's see what we got. Can we find Uithoflijn?
for d in dossiers:
    for fn in d['filenames']:
        if 'ithoflijn' in fn:
            if True:
                print(d['filenames'])
                print(len(set(d['doc_ids'])))
                print(len(set(d['entry_ids'])))
                print(len(set(d['item_ids'])))
                print(len((d['origin_ids'])))
                print(d['origin_types'])
                print(d['origin_ids'])
    #            print(d)
                print()
                print(d)
                print()
                
                
                
            
            print()
#            for time in d['timeline']:
#                print(d['timeline'][time])

#                if d['timeline'][time][1] == 'item':
#                    print(str(item) + ' ' + itemsInformation[d['timeline'][time][0]])
#                else:
#                    print(d['timeline'][time][0])
#                    print(str(item) + ' ' + entriesInformation[d['timeline'][time][0]])
                
        break
        
    
#['Kwartaalrapportage Werkend Tramvervoersysteem Uithoflijn 548041']    has 2 entries and 1 item !!!


['Commissiebrief stand van zaken Uithoflijn 202293']
2
1
1
6
['item', 'item', 'item', 'item', 'item', 'entry']
['87df49bf-b1ff-4f94-ace7-3ebbc16d1409', '87df49bf-b1ff-4f94-ace7-3ebbc16d1409', '87df49bf-b1ff-4f94-ace7-3ebbc16d1409', '87df49bf-b1ff-4f94-ace7-3ebbc16d1409', '87df49bf-b1ff-4f94-ace7-3ebbc16d1409', '3cf9bf98-c1a8-4aed-9669-0615ab3c42c9']

{'filenames': ['Commissiebrief stand van zaken Uithoflijn 202293'], 'doc_ids': ['637627ea-1b95-4837-b483-4a414e2d294b', '637627ea-1b95-4837-b483-4a414e2d294b', '637627ea-1b95-4837-b483-4a414e2d294b', '637627ea-1b95-4837-b483-4a414e2d294b', '637627ea-1b95-4837-b483-4a414e2d294b', '660670a4-7591-417f-9835-ca572fb374e4'], 'item_ids': ['87df49bf-b1ff-4f94-ace7-3ebbc16d1409', '87df49bf-b1ff-4f94-ace7-3ebbc16d1409', '87df49bf-b1ff-4f94-ace7-3ebbc16d1409', '87df49bf-b1ff-4f94-ace7-3ebbc16d1409', '87df49bf-b1ff-4f94-ace7-3ebbc16d1409'], 'entry_ids': ['3cf9bf98-c1a8-4aed-9669-0615ab3c42c9'], 'origin_ids': ['87df49bf-b1ff-4f94-ace7-3ebbc16d1409', '

In [16]:
#why can't we find doc d8539e17-bd18-4850-95a3-397b0233df92
testid = 'd8539e17-bd18-4850-95a3-397b0233df92'
#print(documentsInformation['d8539e17-bd18-4850-95a3-397b0233df92'])
print()
#print its origin
#print(itemsInformation['ff6f90ad-0c73-4b45-84a8-3eabadcdfa4a'])
for item in itemsInformation:
    if testid in itemsInformation[item]['Documents']:
        print('hi')
        
-> We aren't checking literal re-use of id? We aren't checking. We're assuming the titles will be the same,
which will let us find them. However, 

SyntaxError: invalid syntax (Temp/ipykernel_21400/203777526.py, line 11)

In [17]:
#uniqueses has all the duplicate documents. now we want to check out document re-use

entrydocIds = {}
itemdocIds = {}

#test if entries re-use documents with the same id
count = 0
for entry in entriesInformation:
    doc = []
    for d in entriesInformation[entry]['Documents']:
        if d in entrydocIds:
            entrydocIds[d].append(entry)
            count +=1
        else:
            entrydocIds[d] = [entry]

print(count)
count = 0
for item in itemsInformation:
    doc = []
    for d in itemsInformation[item]['Documents']:
        if d in itemdocIds:
            itemdocIds[d].append(item)
            count +=1
        else:
            itemdocIds[d] = [item]

print(count)
print()

entryIds = {}
itemIds = {}

#filter results that are not re-used
for e in entrydocIds:
    if len(entrydocIds[e]) > 1:
        entryIds[e] = deepcopy(entrydocIds[e])
for e in itemdocIds:
    if len(itemdocIds[e]) > 1:
        itemIds[e] = deepcopy(itemdocIds[e])

print('Turns out entries also re-use documents!')
for d in entryIds:
    if len(entryIds[d]) > 5:
        print(entryIds[d])
        print()
        print(listdocumentsInformation[d])
        print()
        


984
12706

Turns out entries also re-use documents!
['17ebd43d-2027-4351-9f86-97d3a54e7e9f', '18fb4e01-88e6-42e5-b33d-bae9d0e388c5', '329f7693-1c27-4b4f-8c44-4c24ab27e44e', '5caa4cee-a455-42fd-a5a7-2c1030cba450', '85d92b70-2dbc-4f45-8727-4ac7a8248285', 'a9e9aa9b-316b-49f5-af40-0e68a2ee92df', 'c63ddb2b-2568-4e0b-9191-37ad985e8bca', 'ceae8360-5215-4d61-8e4d-ead38b970c56', 'd329c577-d5b0-43e2-ae52-f099dbbafcd4']

{'document id': 'a8b094f7-c65f-438b-96e7-ef027a348d91', 'origin id': 'd329c577-d5b0-43e2-ae52-f099dbbafcd4', 'origin type': 'entry', 'displayname': 'CoRa/BGS/Commissiebrieven/2012/Leeswijzer  tekstwijzigingen Kantorenstrategie.pdf', 'filename': 'CoRa/BGS/Commissiebrieven/2012/Leeswijzer  tekstwijzigingen Kantorenstrategie.pdf', 'filesize': '38656', 'publicdownloadurl': 'https://api1.ibabs.eu/publicdownload.aspx?site=Utrecht&id=a8b094f7-c65f-438b-96e7-ef027a348d91'}

['fee563ae-1249-4c45-880f-43bbe5934cda', 'cde28f61-51f6-46aa-800b-3fe7f845ff7d', '05e3280f-64a1-4875-bc08-1b6cf61e9

In [18]:
#Is there overlap between key ids?
for d in documentsInformation:
    if d in listdocumentsInformation:
        print(d)
print('No overlap')

#filter results 

#loop through entrydoclists; add any we don't find
for docid in entryIds:
    found = False
    for i, u in enumerate(udos):
        if docid in u['doc_ids']:
            udos[i]['entry_ids'] = set(list(udos[i]['entry_ids']) + entryIds[docid])
            found = True
    #add new dossier
    if not found:
        #it's important to know what documetns it has!
        newdoc = {   'filenames':[listdocumentsInformation[docid]['filename']],
                     'doc_ids':[docid],
                     'item_ids':[],
                     'entry_ids':entryIds[docid]
                 }            
        
        udos.append(deepcopy(newdoc))

print('halfway')

for docid in itemIds:
    found = False
    for i,u in enumerate(udos):
        if docid in u['doc_ids']:
            udos[i]['item_ids'] = set(list(u['item_ids']) + itemIds[docid])
            found = True
    #add new dossier
    if not found:
#        if docid == testid:
#            print('HERE2')    I guess it does pass here! As expected
        #it's important to know what documetns it has!
        newdoc = {   'filenames':[documentsInformation[docid]['filename']],
                      'doc_ids':[docid],
                     'item_ids':itemIds[docid],
                     'entry_ids':[],
                 }            
        udos.append(deepcopy(newdoc))


print('fini')
#for u in udos:
#    print(len(u['item_ids']))


No overlap
halfway
fini


In [19]:
#going back to previous code to find out: why do some items have entries as itemids?
for d in udos:
    #for i in set(d['item_ids']):
    #    print(itemsInformation[i]['title'])
    
    for i in set(d['entry_ids']):
        if 'title' in entriesInformation[i]:
            print(entriesInformation[i]['title'])
        elif 'Onderwerp' in entriesInformation[i]:
            print(entriesInformation[i]['Onderwerp'])
        elif 'Omschrijving' in entriesInformation[i]:
            print(entriesInformation[i]['Omschrijving'])
        else:
            print(entriesInformation[i])
            print(entriesInformation[i]['title'])

Commissiebrief Openbare versie brief 7 mei 2013 gerechtelijke procedure Pythagoraslaan
Cie S&R:  nadere toegezegde info inzake BP De Trip
Raadsbrief: Toezeggingen uitwerkingsnota (verzonden26-9-2014)
Raadsbrief Voorhangprocedure bilaterale ontwikkelovereenkomst Jaarbeurspleingebouw (verzonden 27082014)
Raadsbrief Voorhangprocedure bilaterale ontwikkelovereenkomst Jaarbeurspleingebouw (verzonden 27082014)
Raadsbrief Voorhangprocedure bilaterale ontwikkelovereenkomst Jaarbeurspleingebouw (verzonden 27082014)
Raadsbrief Voorhangprocedure bilaterale ontwikkelovereenkomst Jaarbeurspleingebouw (verzonden 27082014)
Raadsbrief Voorhangprocedure bilaterale ontwikkelovereenkomst Jaarbeurspleingebouw (verzonden 27082014)
Cie S&R: Welstandsvrije bouwwerken en gebieden / welstandsadvisering door supervisoren
Cie S&R: voortgangsrapportage Utrechtse Energie
Cie S&R: voortgangsrapportage Utrechtse Energie
Uitwerken maatregelen ter vervanging van knip Monicabrug
Uitwerken maatregelen ter vervanging van

SV 2021 nr. 81 Jaffastraat 
SV 2021 nr. 84 Subsidie voor posteractie KRU 
SV 2021 nr. 84 Subsidie voor posteractie KRU 
Verslag Vragenuur, 2017-10-12
Verslag Vragenuur, 2017-10-12
Verslag Vragenuur, 2019-11-28
Verslag Vragenuur, 2019-11-28
Verslag Vragenuur, 2019-04-11
Verslag Vragenuur, 2019-04-11
Verslag Vragenuur, 2019-09-19
Verslag Vragenuur, 2019-09-19
Verslag Vragenuur, 2018-04-26
Verslag Vragenuur, 2018-04-26
Verslag Vragenuur, 2017-01-26
Verslag Vragenuur, 2017-01-26
Verslag Vragenuur, 2016-12-22
Verslag Vragenuur, 2016-12-22
Verslag Vragenuur, 2017-01-12
Verslag Vragenuur, 2017-01-12
Verslag Vragenuur, 2016-12-15
Verslag Vragenuur, 2016-12-15
Verslag Vragenuur, 2019-09-12 
Verslag Vragenuur, 2019-09-12 
Verslag Vragenuur, 2019-11-07 
Verslag Vragenuur, 2019-11-07 
Verslag Vragenuur, 2017-01-19
Verslag Vragenuur, 2017-01-19
Verslag Vragenuur, 2019-10-17
Verslag Vragenuur, 2019-10-17
Verslag Vragenuur, 2017-10-05
Verslag Vragenuur, 2017-10-05
Verslag Vragenuur, 2019-10-10
Versla

M 117 Wandelroutes in de binnenstad én daarbuiten
M 117 Wandelroutes in de binnenstad én daarbuiten
M128 Atelierwoningen voor kunstenaars
M128 Atelierwoningen voor kunstenaars
M130 Sociale Huur in het Stationsgebied
M130 Sociale Huur in het Stationsgebied
M126 informeren over maatwerk
M126 informeren over maatwerk
M131 Monitoren effecten parkeerbeleid Cartesiusdriehoek
M131 Monitoren effecten parkeerbeleid Cartesiusdriehoek
M123  Hand in hand met de ondernemer tegen overlast
M123  Hand in hand met de ondernemer tegen overlast
M127 Sociale Huur in het Stationsgebied
M127 Sociale Huur in het Stationsgebied
M124  24 uur bezetting politiepost Leidsche Rijn en meer Boa's
M124  24 uur bezetting politiepost Leidsche Rijn en meer Boa's
M134 niet over ons zonder ons
M134 niet over ons zonder ons
M133 Proef onderbord bij bord E 06 RVV (gehandicaptenparkeerplaats
M133 Proef onderbord bij bord E 06 RVV (gehandicaptenparkeerplaats
M137 toegankelijkheid
M137 toegankelijkheid
M125 Maximum aan maatwer

In [20]:
#Investigate why itemIds takes so long
#for docid in itemIds:

#Retrying previous tests

#why can't we find doc d8539e17-bd18-4850-95a3-397b0233df92
# https://ureka.utrecht.nl/app/instrument/raadsvoorstel-9f9aed03-b56b-48f6-b41e-36d205551378
testid = 'd8539e17-bd18-4850-95a3-397b0233df92'
print(itemIds[testid])
#print(udos[testid])

for d in udos:
    if testid in d['doc_ids']:
        print('hi')
        
print('found?\n')

uithofset = []
    
#Can we find Uithoflijn?
for i, d in enumerate(udos):
    if not('filenames' in d):
        udos[i]['filenames'] = [documentsInformation[d['doc_ids'][0]]['filename']]
        
    for fn in d['filenames']:
        if 'ithoflijn' in fn:
            uithofset.append(deepcopy(d))
            if testid in d['doc_ids']:
                print(d)
            
            if True:
                print()
                print(d['doc_ids'])
                print(d['filenames'])
                
                tittle = 'title'
                
                if 'doc_ids' in d:
                    print('docs ' + str(len(set(d['doc_ids']))))
                if 'entry_ids' in d:
                    print('entries ' + str(len(set(d['entry_ids']))))
                    for e in set(d['entry_ids']):
#                        print(d)
                        if 'Onderwerp' in entriesInformation[e]:
                            print(entriesInformation[e]['Onderwerp'])
                            tittle = 'Onderwerp'
                        else:
                            print(entriesInformation[e]['title'])
                if 'item_ids' in d:
                    print('items ' + str(len(set(d['item_ids']))))
                    for i in set(d['item_ids']):
                        #print(i)
                        #print(itemsInformation[i])
                        if 'Onderwerp' in itemsInformation[i]:
                            print(itemsInformation[i]['Onderwerp'])
                            tittle = 'Onderwerp'
                        else:
                            print(itemsInformation[i]['title'])
                
                #print('timeline ' + str(len(d['timeline'])))
                    
                    
#TODO checken we nu echt transitief?
#STEL je hebt dus twee documenten die beide ergens anders gecite worden
# worden die altijd gelinkt? wel in normal usecase
# ook als het twee verschillende soorten dupes zijn? ik denk het wel..

#kijk naar voorbeelden: waarom wordt deze niet verder gelinkt?


#Bijlage Uithoflijn van raadslid Van Schie 212398   heeft twee agendapunten
#Rekenkameronderzoek: ‘Samen sturen. Onderzoek naar de bestuurlijke en ambtelijke aansturing voor de realisatie van de Uithoflijn’
#Rekenkameronderzoek: ‘Samen sturen. Onderzoek naar de bestuurlijke en ambtelijke aansturing voor de realisatie van de Uithoflijn’
#docids ['5feb0010-ddac-4caf-8bcc-8ed4d4a2690a', '432fcf54-9901-47d7-94ce-428e48f4da94', '5feb0010-ddac-4caf-8bcc-8ed4d4a2690a', '432fcf54-9901-47d7-94ce-428e48f4da94', '5feb0010-ddac-4caf-8bcc-8ed4d4a2690a', '432fcf54-9901-47d7-94ce-428e48f4da94']


#Dit heeft overlap met raadsvoorstel
#['d8539e17-bd18-4850-95a3-397b0233df92']
#['20181204 Raadsvoorstel Uithoflijn.pdf']
#RV Rekenkameronderzoek: ‘Samen sturen. Onderzoek naar de bestuurlijke en ambtelijke aansturing voor de realisatie van de Uithoflijn’




['c583af2b-088c-43c6-bf2e-c40d15e17d7c', '393b9725-db47-43d7-bf43-bfadbe744380', '9f9aed03-b56b-48f6-b41e-36d205551378', '259ae3aa-8cec-46c7-9ee3-b7589318dbe5', 'd51ace51-fd3f-4955-a797-20505bab8b63', 'ff6f90ad-0c73-4b45-84a8-3eabadcdfa4a']
hi
found?


['637627ea-1b95-4837-b483-4a414e2d294b', '637627ea-1b95-4837-b483-4a414e2d294b', '637627ea-1b95-4837-b483-4a414e2d294b', '637627ea-1b95-4837-b483-4a414e2d294b', '637627ea-1b95-4837-b483-4a414e2d294b', '660670a4-7591-417f-9835-ca572fb374e4']
['Commissiebrief stand van zaken Uithoflijn 202293']
docs 2
entries 1
Commissiebrief S&R: Stand van zaken Uithoflijn
items 1
Cie S&R: stand van zaken Uithoflijn

['da4c74f5-f5fb-41c7-9201-298e30731e3f', 'da4c74f5-f5fb-41c7-9201-298e30731e3f', 'da4c74f5-f5fb-41c7-9201-298e30731e3f', 'da4c74f5-f5fb-41c7-9201-298e30731e3f', 'da4c74f5-f5fb-41c7-9201-298e30731e3f', 'e86cad51-92b7-402f-9687-b28d05da5553']
['Commissiebrief Ontwikkelingen Uithoflijn 119820']
docs 2
entries 1
Commissiebrief S&R: Ontwikkelingen

In [21]:
#merge timelines with overlapping entries or items  - DID WE JUST DO ONE ITERATION?
#for d in udos:
#    for d2 in udos:
#        if d != d2:
#            for e in d['entry_ids']:
#                if e in d2['entry_ids']:
#                    print('merge')

ents = {}
its = {}
#print('lets make a map from entries to udos')
for i, d in enumerate(udos):
    for e in d['entry_ids']:
        if e in ents:
            ents[e].append(i)
        else:
            ents[e] = [i]
    for e in d['item_ids']:
        if e in its:
            its[e].append(i)
        else:
            its[e] = [i]

print(len(ents))
print(len(its))

for e in ents:
    if len(ents[e]) > 10:
        if 'title' in entriesInformation[e]:
            print(str(len(ents[e])) + ' ' + str(e) + ' ' + entriesInformation[e]['title'])
        elif 'Onderwerp' in entriesInformation[e]:
            print(str(len(ents[e])) + ' ' + str(e) + ' ' + entriesInformation[e]['Onderwerp'])
        elif 'Omschrijving' in entriesInformation[e]:
            print(str(len(ents[e])) + ' ' + str(e) + ' ' + entriesInformation[e]['Omschrijving'])
        else:
            print(e)
        
print()

for e in its:
    if len(its[e]) > 10:
        if 'title' in itemsInformation[e]:
            print(str(len(its[e])) + ' ' + str(e) + ' ' + itemsInformation[e]['title'])
        else:
            print(e)

#testid = '26e342f0-8947-480a-9622-d51f79d29f94'
#if testid in itemsInformation:
#    print('1')
#if testid in entriesInformation:
#    print('2')

#todo: items disambiguation

3645
6575
12 f66eaf17-03de-42dc-8482-0640648d21cb Cie S&R: Commissiebrief Gebiedsagenda's Utrecht Aantrekkelijk en Bereikbaar (UAB)
13 0e5d7350-6ab0-4edb-92f7-ebfa1a1b45c4 Commissiebrief S&R: Bestemmingsplan Kanaalweg 59 / Wilhelminawerf
11 a646fb18-7a22-47e5-a750-129a93749b92 Raadsbrief Ontwikkeloverenkomst Jaarbeursterrein 16 maart 2016
23 3297399d-a5e1-4a61-8980-c8ce3e9bb7f5 Raadsbrief Prestatieafspraken 2017-2020
13 31d29a49-6a03-4242-b4e4-39b238547bd8 Raadsbrief Evaluatie Initiatievenfonds
14 f576f990-0a3c-463d-a3e5-8501120d9fe5 Raadsbrief Luchtkwaliteit en gezondheid
11 26e342f0-8947-480a-9622-d51f79d29f94 SV 2018, nr. 26 over Vervolgdebat over de Uithoflijn op 8 maart 2018 (met antwoord)
14 37eb47f4-2b1a-4e21-9348-39a5b15c38d7 Raadsbrief Beantwoording motie vervolgstappen Bewonersbod
15 31831eef-419c-4cc3-a40f-7b99cb8622d0 Memo Aanvullende informatie Nieuwe Zandpad naar aanleiding van RIB 5 september
18 63f23fdf-8e42-4add-98e8-27240b20a531 Raadsbrief Voorhangprocedure oprichting

In [22]:
#so we should merge udos when they have the same entries!
udos2 = []


#we're gonna give up on the timeline for now... we'll repair that after the fact
def mergeDos(d, ud):
    #print('merging ' + str(d['filenames']) + '  with  ' + str(ud['filenames']))
    #print(d)
    ud['filenames'] = set(list(ud['filenames']) + list(d['filenames']))
    ud['doc_ids'] = set(list(ud['doc_ids']) + list(d['doc_ids']))
    ud['item_ids'] = set(list(ud['item_ids']) + list(d['item_ids']))
    ud['entry_ids'] = set(list(ud['entry_ids']) + list(d['entry_ids']))
    
    #ud['origin_ids'] = ud['origin_ids'] + d['origin_ids']
    #ud['origin_types'] = ud['origin_types'] + d['origin_types']
    
    #for time in d['timeline']:
    #    if time in ud['timeline']:   
    #        for t in d['timeline'][time]:
    #            ud['timeline'][time].append(t)
    #    else: 
    #        ud['timeline'][time] = deepcopy(d['timeline'][time])
    return ud
    
#returns True if at least one dossier was merged
def resolveLinks(udos, udos2):
    merged = 0
    unfound = 0
    for i, dos in enumerate(tqdm(udos)):
        found = False
        for j, dos2 in enumerate(udos2):
            entrymerge = 0
            for e in dos2['entry_ids']:
                if e in dos['entry_ids']:
                    #print('before ' + str(len(udos2)))
                    dos2[j] = mergeDos(dos, udos2[j])
                    #print('after ' + str(len(udos2)))
                    found = True
                    merged += 1
                    entrymerge += 1
#                    break
#            if merged < 1:

            itemmerge = 0
            for i in dos2['item_ids']:
                    
                if i in dos['item_ids']:
                    udos2[j] = mergeDos(dos, udos2[j])
                    itemmerge += 1
                    found = True
                    merged += 1
#                        break
                    
        if found == False:
            unfound += 1
            udos2.append(deepcopy(dos))
            
    return merged

#might have to do this multiple times, because a new dossier can link 2 existing dossiers
#and instead will only be merged with the first
m = 1
udos3 = deepcopy(udos)  #we use a copy of udos because we change it in place
while(m > 0):
    print(m)
    m = resolveLinks(udos3, udos2)#udos
    print(len(udos3))
    print(len(udos2))
    udos3 = deepcopy(udos2)
    udos2 = []
    
print(len(udos))
print(len(udos2))
print(len(udos3))
print(m)




#TODO timeline bij entryIds en listIds

1


100%|██████████████████████████████████████████████████████████████████████████| 10151/10151 [00:08<00:00, 1169.05it/s]


10151
3021
24873


100%|████████████████████████████████████████████████████████████████████████████| 3021/3021 [00:01<00:00, 1563.70it/s]


3021
3004
215


100%|████████████████████████████████████████████████████████████████████████████| 3004/3004 [00:01<00:00, 1588.70it/s]


3004
3004
10151
0
3004
0


In [26]:
uithofset = []
    
def getItemDate(iid):
    item = itemsInformation[iid]
    return meetingsInformation[item['origin id']]['meetingdate']
    
def getEntryDate(eid):
    entry = entriesInformation[eid]
    key = ''
    if 'Datum' in entry:
        key = 'Datum'
    elif 'Datum invoer' in entry:
        key = 'Datum invoer'
    else: 
        key = 'Ontvangstdatum'

    return entry[key]
    
#Can we find Uithoflijn?
for d in udos3:
    if not('filenames' in d):
        d['filenames'] = [documentsInformation[d['doc_ids'][0]]['filename']]
        

    for fn in d['filenames']:
        if 'tatenleden' in fn:
            print()
            print()
            print()
            print()
            meets = []
            dates = []
            ents = []
            its = []      
            
            uithofset.append(deepcopy(d))
            if testid in d['doc_ids']:
                print(d)
            
            if True:
                print()
                print(d['doc_ids'])
                print(d['filenames'])
                
                tittle = 'title'
                
                if 'doc_ids' in d:
                    print('docs ' + str(len(set(d['doc_ids']))))
                if 'entry_ids' in d:
                    print('entries ' + str(len(set(d['entry_ids']))))
                    for e in set(d['entry_ids']):
#                        print(d)
                        if 'Onderwerp' in entriesInformation[e]:
                            tittle = 'Onderwerp'
                        else:
                            tittle = 'title'
                        print(getEntryDate(e) + ' ' + entriesInformation[e][tittle])
                        print('https://utrecht.bestuurlijkeinformatie.nl/Agenda/Index/' + e)
                        print()
                        ents.append((entriesInformation[e], entriesInformation[e][tittle])) #['origin id']
                            
                if 'item_ids' in d:
                    print('items ' + str(len(set(d['item_ids']))))
                    for i in set(d['item_ids']):
                        #print(i)
                        #print(itemsInformation[i])
                        if 'Onderwerp' in itemsInformation[i]:
                            tittle = 'Onderwerp'
                        else:
                            tittle = 'title'
                        #print(itemsInformation[i])
                        print(getItemDate(i) + ' ' + itemsInformation[i][tittle])
                        print('https://utrecht.bestuurlijkeinformatie.nl/Agenda/Index/' + itemsInformation[i]['origin id'])
                        print()
                        its.append((itemsInformation[i]['origin id'], itemsInformation[i][tittle])) 
                        meets.append(itemsInformation[i]['origin id'])
                        dates.append(getItemDate(i))
                        
                #print('timeline ' + str(len(d['timeline'])))
                print()
                print('We found ' + str(len(set(meets))) + ' meetings')
                #for m in meets:
                #    print(meetingsInformation[m]['title'])
                print(set(meets))
                print(set(dates))
                break
        

print('\n\n   Which dossiers we should merge:')
for i in uithofset:
    for j in uithofset:
        if i != j:
            
            for e in i['entry_ids']:
                if e in j:
                    print('merge')
                    
                    
                    
#{'7ac6074f-6930-4357-96cc-dc156a149dd9', '18e58f2a-045d-49d1-bf76-5c08d6131b72'}
#{'Memo Documenten Uithoflijn 1e tranche.docx 126528', 'Memo Documenten Uithoflijn 1e tranche 126528'}
#docs 2 
#entries 2
#Mar  2 2018 12:00AM 3ae1a093-715e-4786-9ea1-58a8bad973c7 itemid Memo Documenten Uithoflijn 1e tranche
#Feb 28 2018 12:00AM 26e342f0-8947-480a-9622-d51f79d29f94 SV 2018, nr. 26 over Vervolgdebat over de Uithoflijn op 8 maart 2018 (met antwoord)
    # Waarom vind hij dan niet 1ab6215c-6451-4ce8-986d-753266a391ce


# voorbeeld hierboven
# wij linken nu 
# https://utrecht.bestuurlijkeinformatie.nl/Reports/Item/3ae1a093-715e-4786-9ea1-58a8bad973c7
# https://utrecht.bestuurlijkeinformatie.nl/Reports/Item/26e342f0-8947-480a-9622-d51f79d29f94
# 
# ureka doet hier niets mee 
# https://ureka.utrecht.nl/app/instrument/3ae1a093-715e-4786-9ea1-58a8bad973c7
#
# note: waarom vinden we niet 
# https://utrecht.bestuurlijkeinformatie.nl/Reports/Item/1ab6215c-6451-4ce8-986d-753266a391ce


#voorbeeld 2
# Sommige documenttypen lijken niet gemodelleerd te worden in de bestaande instrumenten, vooral items uit de Entries/Reports
# Zoeken naar instrumenten lijkt
#https://utrecht.bestuurlijkeinformatie.nl/Reports/Item/119c3915-63e4-4c01-bfef-c582a2c7dddf
#https://utrecht.bestuurlijkeinformatie.nl/Reports/Item/799c5a56-88b9-4e48-8954-1da2b7927873
#https://utrecht.bestuurlijkeinformatie.nl/Reports/Item/1efa6afc-e504-453d-ba3b-e95ce0a169f9

# is dit een goede usecase? https://ureka.utrecht.nl/app/instrument/1329d671-81d4-43dd-bd40-c7133f7b26b0
# https://utrecht.bestuurlijkeinformatie.nl/Reports/Item/1329d671-81d4-43dd-bd40-c7133f7b26b0
#-> nah geen overlap

#voorbeeld3
# https://utrecht.bestuurlijkeinformatie.nl/Reports/Item/a11b9019-1500-44e0-aa71-bd386bc84811
# https://utrecht.bestuurlijkeinformatie.nl/Reports/Item/af4abff9-2534-40d3-8c4c-403a306e836e





#andres dan ureka
#2018-11-01T19:30:00 c3413368-5d5d-4267-b15d-23308abc9a0c Voorstel Deelname Statenleden aan beraadslaging
#2018-10-04T14:00:00 9ad8e1c2-6b68-4329-b778-7b1fd9dd4231 Informatie- en overlegstatuut Raad Utrecht en Staten Utrecht m.b.t. het project Uithoflijn 
#2018-10-11T14:00:00 0d1dea5a-6f81-46a3-8f27-c682a42dc56c Informatie- en overlegstatuut Raad Utrecht en Staten Utrecht m.b.t. het project Uithoflijn 
#2018-10-04T20:00:00 967aa19f-2f64-4e34-a52f-be473cc46154 Informatie- en overlegstatuut Raad Utrecht en Staten Utrecht m.b.t. het project Uithoflijn 






{'08fd7282-2d83-4316-8eda-2bd7f1b623f6', 'fcca33d0-a6a4-4a06-b7f6-ea5f4a164844', '6bc95466-2edd-42a6-8846-904f4fb3a189', '8b9adfda-d2f0-48dc-b9ec-9789371786af', '2f468205-de92-4ffe-98c6-7a1c10e2e547', 'ca6d3115-15c9-4c79-bd91-d329a81b281f'}
{'Dossier 336 voorblad 68000', 'Memo_Memo informatie- en overlegstatuut Raad Utrecht en Staten Utrecht m.b.t. het project Uithoflijn 94768', 'Bijlage 2; Voorstel_Statenleden deelnemen aan de beraadslaging.pdf', 'Bijlage 1; Verslag bijeenkomst PSraadUtrecht2706.pdf'}
docs 6
entries 0
items 4
2018-10-04T20:00:00 Informatie- en overlegstatuut Raad Utrecht en Staten Utrecht m.b.t. het project Uithoflijn 
https://utrecht.bestuurlijkeinformatie.nl/Agenda/Index/967aa19f-2f64-4e34-a52f-be473cc46154

2018-11-01T19:30:00 Voorstel Deelname Statenleden aan beraadslaging
https://utrecht.bestuurlijkeinformatie.nl/Agenda/Index/c3413368-5d5d-4267-b15d-23308abc9a0c

2018-10-11T14:00:00 Informatie- en overlegstatuut Raad Utrecht en Staten Utrecht m.b.t. het proj

In [296]:
name1 = 'Beantwoording SV 2017 nr 69'
name2 = 'Raadsbrief Voortgang vergunningen en ontheffingen nav moties 2016 96 en 121'
name3 = 'Jaarstukken 2016'
name4 = 'Voorstel_11667'

for d in udos3:
    for fn in d['filenames']:
        if name4 in fn:
            print(d['filenames'])
            break


# 61c08ca4-2f57-474b-8a7a-0b398e091ccb  - verzameling schriftelijk vragne antwoorden    type 0     geen hoofdpunt   !
# fd864cf8-6ecc-4c73-825d-fcc0a848a27f  - idem      !
# 94e21e8f-6977-4883-9aca-403ba7f13f10  idem


# d2eff14d-7735-4e88-83a6-ce11b1b8d76f   jaarstukken   type 2  !
# 94e21e8f-6977-4883-9aca-403ba7f13f10     idem twijfelgeval
# 20ecac3a-b8f5-4817-b7c9-d55904e7a5d1
# a1b22807-377a-46b8-bd23-9afd5ba3540b

# 45aac33c-d537-47e4-af63-1f1c1edfd359  voorjaarsnota !!!

# 429140f0-24d2-4ca5-a83b-f3577baa29cc  vragenuur type 17    !
# c2b2c62c-e387-4723-a19f-3e4a1cfbad85  vragenuur type 17


#b8bb8994-3f4a-4f70-b8f4-8d4901a7ea01   twijfelgeval. verzamelt armoede aanpak dingen. handig als je 

#ea94b4ad-ee17-447d-aa25-81fd1aea8bdf
# 17977cce-f828-400f-84cc-958dd5ba896e   twijfel

#9fa8e966-afc5-4d64-8f4f-e7fe9301ffba   stemming 100716987 
#3e0ed66f-462c-46f0-98d5-dbb4948e57dc    type 100716987   stemagenda raad 
#f8b3c018-31ae-47ec-b7f3-2f2f7def73f0  100716987 
#itemid 6.1.1 55473b4b-31fb-45ab-a677-1b7124d52060 100716987
#a8e676cf-e88c-4999-854b-3ebd77f185b6  100716987
#0a997446-f6ab-4d59-8695-ac634dae0284 100716987

{'Besluitenhistorie Nota Kapitaal Goederen 2020 - 2023 48506', 'Meerjaren Perspectief Bereikbaarheid (juni 2018) 10279228', 'Motivering niet-toepasselijkheid referendumverordening MPUV 2018 28356', 'Voorstel_11977 402058', 'Motivering niet-toepasselijkheid referendumverordening Voorjaarsnota 2018 en Eerste Bestuursrapportage 2018 28774', '11 Bijlage Fietsparkeerbalans Stationsgebied 2432854', 'Motivering niet-toepasselijkheid Verordening tot raadgevend referendum 28768', 'Jaarstukken 2017 6111507', 'Motivering niet-toepasselijkheid Referendumverordening 27816', 'Dossier 682 voorblad 18704', 'Nota Kapitaalgoederen Vastgoed 2020-2023 (juli 2019) 1757652', 'Dossier 581 voorblad.pdf', 'Voorstel_11584 267554', 'Voorstel_11667 277268', 'Dossier 262 voorblad 70704', '1 Aanbiedingsbrief Meerjarenperspectief Ruimte 2019 56298', 'MPUV 2019 - Meerjaren Perspectief Utrechts Vastgoed 928146', 'w.g. Accountantsverslag 2017 gemeente Utrecht.pdf', 'Raadsbrief Projectenlijst Nota Kapitaal Goederen 3418

In [30]:
piratne

Mar  2 2018 12:00AM 3ae1a093-715e-4786-9ea1-58a8bad973c7 Memo Documenten Uithoflijn 1e tranche
    zou in beide moeten zitten! waarom zien we hem alleen in entries?


Bijlage 2 - Chronologisch overzicht bestuurlijk traject Uithoflijn 182000
Raadsbrief Ter inzage legging stukken Uithoflijn 273482
Raadsbrief Ter inzage legging stukken Uithoflijn 273482
Bijlage bij raadsbrief - Statenbrief over Uithoflijn (2 maart 2018) 377406
Bijlage bij raadsbrief - Statenbrief over Uithoflijn (2 maart 2018) 377406
Memo Documenten Uithoflijn 1e tranche 126528
Bijlage 3 Mandaat projectdirecteur Uithoflijn_Geredigeerd2 248757
Bijlage 4 Notitie Mandaatregeling project Uithoflijn 2012_Geredigeerd 55678
Ingekomen SV 2019 _19_ Uithoflijn 697616
Beantwoording SV 2019 nr 182 over Uitstel Uithoflijn 188177
Raadsbrief Tweede kwartaalrapportage 2018 en aangevulde bestuursovereenkomst Uithoflijn 161104
Kwartaalrapportage Q2 2018 Werkend Tramvervoersysteem Uithoflijn 1330717
Allonge II Bestuursovereenkomst Uithoflijn + openbare bijlagen 1 en 2 1380727


print('hi')

#TODO sub-agendapunten!
#nu houden zowel Ureka als ik ze apart, terwijl dit voorbeeld de kracht laat zien
#https://utrecht.bestuurlijkeinformatie.nl/Agenda/Index/6033aaab-fee5-416b-bf65-46b993cb6e40#86a46282-d1a9-4528-a6a9-9cd4c146cb8f
    
#    https://ureka.utrecht.nl/app/instrument/5384b773-4317-4511-a73b-cf0a68de90e7
#    https://ureka.utrecht.nl/app/instrument/raadsvoorstel-9f9aed03-b56b-48f6-b41e-36d205551378

#waarom vind ik Memo_Memo informatie- en overlegstatuut Raad Utrecht en Staten Utrecht m.b.t. het project Uithoflijn.pdf   niet?
fcca33d0-a6a4-4a06-b7f6-ea5f4a164844', '08fd7282-2d83-4316-8eda-2bd7f1b623f6', '6bc95466-2edd-42a6-8846-904f4fb3a189', '8b9adfda-d2f0-48dc-b9ec-9789371786af', '2f468205-de92-4ffe-98c6-7a1c10e2e547'}
{'Bijlage 1; Verslag bijeenkomst PSraadUtrecht2706.pdf', 'Bijlage 2; Voorstel_Statenleden deelnemen aan de beraadslaging.pdf', 'Memo_Memo informatie- en overlegstatuut Raad Utrecht en Staten Utrecht m.b.t. het project Uithoflijn.pdf', 'Dossier 336 voorblad 68000'}
docs 5
entries 0
items 3
Informatie- en overlegstatuut Raad Utrecht en Staten Utrecht m.b.t. het project Uithoflijn 
Informatie- en overlegstatuut Raad Utrecht en Staten Utrecht m.b.t. het project Uithoflijn 
Informatie- en overlegstatuut Raad Utrecht en Staten Utrecht m.b.t. het project Uithoflijn 



{'51826fd1-e959-4550-9e59-7441f1409ced', '0f95b5ab-bbe9-42cd-9a72-74a2dd524f10', '213bcf1b-4ade-4c31-9d97-4afad5352fc8', 'a55ab0a7-7ab3-487e-85fd-9dde3574aff6', '590dfd27-34c7-45fe-8ee5-7fcd0d6bccad', '503e48ef-739f-414e-bf2d-f213cf350f88', 'e53670e6-fce5-440c-bbfd-6824d8a08106', '7f221d04-41b1-4e97-9cf4-e12ba670ed50', '5966cf2a-b9ab-408a-b692-1245636013ec'}
{'Dossier 457 voorblad 68176', 'Bijlage 2_ Procesbeschrijving VTWmanagement_ POUHL_Geredigeerd.pdf', 'Beantwoording SV 2019 nr. 19 inzake de Uithoflijn.pdf', 'Ingekomen SV 2019 _19_ Uithoflijn.pdf', 'Bijlage 1 _Toelichting bij beantwoording vraag 4 - SV 2019 - nr 19.pdf', 'Dossier 457 voorblad 66688', 'Bijlage 4_ Notitie Mandaatregeling project Uithoflijn 2012_Geredigeerd.pdf', 'Bijlage 3_ Mandaat projectdirecteur Uithoflijn_Geredigeerd2.pdf'}
docs 9
entries 0
items 3
SV 2019, nr. 19 inzake de Uithoflijn
SV 2019, nr. 19 inzake de Uithoflijn
SV 2019, nr. 19 inzake de Uithoflijn


In [ ]:
TODO hoe relateren tijdlijnen
                        if x['origin type'] == 'item':
                        dossier['item_ids'].append(x['origin id'])

                        item = itemsInformation[x['origin id']]
                        #print(meetingsInformation[item['origin id']]['meetingdate'])

                        #NOTE meetingdate and publishdate are different
                        #publishdate seems more accurate with test data (origin publish instead of re upload date)

                        if meetingsInformation[item['origin id']]['meetingdate'] in dossier['timeline']:
                            dossier['timeline'][meetingsInformation[item['origin id']]['meetingdate']].append([x['origin id'], 'item'])
                        else:
                            dossier['timeline'][meetingsInformation[item['origin id']]['meetingdate']] = [[x['origin id'], 'item']]
                        #for doc in item['Documents']:
                        #    dossier['doc_ids'].append(doc)

                    elif x['origin type'] == 'entry':
                        dossier['entry_ids'].append(x['origin id'])

                        entry = entriesInformation[x['origin id']]

                        #print(entry)
                        key = ''
                        if 'Datum' in entry:
                            key = 'Datum'
                        elif 'Datum invoer' in entry:
                            key = 'Datum invoer'
                        else: 
                            key = 'Ontvangstdatum'

                        if entry[key] in dossier['timeline']:
                            dossier['timeline'][entry[key]].append([x['origin id'], 'entry'])
                        else:
                            dossier['timeline'][entry[key]] = [[x['origin id'], 'entry']]
                        #dossier['doc_ids'].append(doc)

                    else:
                        print('Origin type')
                        print(x)
                        
                        
                        
                        #this method only counts the number of duplicates based on a matching title
TODO agendapunten


#TODO statistics on duplicates
    #How much within each document set?
    #How much between document sets
    #HOW MANY DUPLICATES - are we going to find many timemlines?

#TODO differences


def deduplicate(udocs):
    uniquedocs = {}
    for filename in udocs:
        for doc in udocs[filename]:
            
    
            return uniques


for doc in uniquedocs:
    l = len(uniquedocs[doc])
    if l > 10:
        print(doc)
        #for d in uniquedocs[doc]:
        #    print(d['filesize'] + '   ' + d['displayname'])
        print()

#test = list(uniquedocs)[:10]
#print(test)
#print(len(uniquedocs))


#Test If filename = same and displayname - pdf is same  
    #EXCEPTION is 
    

Beleidsdocumenten van de Omgevingsvisie Utrecht 1.pdf
Beleidsdocumenten van de Omgevingsvisie Utrecht 1.0 (december 2017)
Beleidsdocumenten van de Omgevingsvisie Utrecht 1.0 (december 2017)
Beleidsdocumenten van de Omgevingsvisie Utrecht 1.0
Beleidsdocumenten van de Omgevingsvisie Utrecht 1.0 (september 2017)
Beleidsdocumenten van de Omgevingsvisie Utrecht 1.pdf
Beleidsdocumenten van de Omgevingsvisie Utrecht 1.pdf

# Duplicate agenda item titles